In [36]:
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from pathlib import Path

def import_workout_csv(csv_path: str, db_path: str = "./hr_data/database.duckdb"):
    # ------------------------------------------------------------------
    # 1.  Read the CSV and build the workoutId
    # ------------------------------------------------------------------
    print(f"Importing csv file {csv_path} to database {db_path}")
    csv_file = Path(csv_path)
    
    # -- first row holds file-wide metadata
    metadata_df = pd.read_csv(csv_path, nrows=1)
    
    # create "yyyy-mm-dd_HHMMSS" style id (remove “:” so it is filename-safe)
    metadata_df["workoutId"] = (
        metadata_df["Date"].astype(str).str.strip() + "_" +
        metadata_df["Start time"].astype(str).str.replace(":", "", regex=False).str.strip()
    )
    #print(metadata_df["workoutId"].iloc[0]);

    # ------------------------------------------------------------------
    # 2.  Open DuckDB and make sure tables exist
    # ------------------------------------------------------------------
    con = duckdb.connect(db_path)
    try:
        # ------------------------------------------------------------------
        # 3.  Insert metadata only if the workoutId is new
        # ------------------------------------------------------------------
        con.register("meta_view", metadata_df)
        con.execute("""
        CREATE TABLE IF NOT EXISTS workout_metadata AS
        SELECT * FROM meta_view LIMIT 0;           -- schema only
        """)
        
        result = con.execute("""
        INSERT INTO workout_metadata
        SELECT *
        FROM meta_view AS m
        WHERE NOT EXISTS (
            SELECT 1 FROM workout_metadata w WHERE w.workoutId = m.workoutId
        );
        """)
        print(f"\tNumber of inserted rows in workout matadata: {result.rowcount}.")
        
        # ------------------------------------------------------------------
        # 4.  Quick sanity check
        # ------------------------------------------------------------------
        #print("Latest workout metadata:")
        #display(con.execute("SELECT * FROM workout_metadata ORDER BY Date DESC, \"Start time\" DESC LIMIT 1").fetchdf())
    
        
        # ------------------------------------------------------------------
        # 5.  Read the time-series rows, add FK to link back to metadata,
        #     and append them to the timeseries table
        # ------------------------------------------------------------------
        df = pd.read_csv(csv_path, skiprows=2)
        
        # keep link to its parent workout
        df["workoutId"] = metadata_df.at[0, "workoutId"]
        
        con.register("ts_view", df)
        con.execute("""
        CREATE TABLE IF NOT EXISTS timeseries AS
        SELECT * FROM ts_view LIMIT 0;           -- schema only
        """)
        
        result = con.execute("""
        INSERT INTO timeseries
        SELECT * FROM ts_view;
        """)
        print(f"\tNumber of inserted rows in workout timeseries: {result.rowcount}.")
        print("\tImported.")
    
        # ------------------------------------------------------------------
        # 6.  Quick sanity check
        # ------------------------------------------------------------------
        #print("\nFirst five time-series rows:")
        #display(con.execute("SELECT * FROM timeseries WHERE workoutId = ? LIMIT 5",
        #                  [metadata_df.at[0, "workoutId"]]).fetchdf())

    except Exception as e:
        print("❌ Error while importing workout CSV:")
        print(e)

    finally:    
        con.close()

In [37]:
# ------------------------------------------------------------------
# 6.  Scans the directory for all svc files
# ------------------------------------------------------------------
data_dir = Path("./hr_data").resolve()

# Get all CSV files starting with 'Anton_Antonov' (case-sensitive)
csv_files = sorted(data_dir.glob("Anton_Antonov*.CSV"))

if not csv_files:
    print("No workout CSV files found.")
else:
    print(f"Found {len(csv_files)} CSV file(s). Importing...\n")
    
    for csv_path in csv_files:
        import_workout_csv(str(csv_path))
        new_path = csv_path.with_name(f"Imported_{csv_path.name}")
        csv_path.rename(new_path)

No workout CSV files found.
